# 다층신경망(MLP)

## 1) 기본 Package 설정

In [ ]:
# 기본
import numpy as np
import matplotlib.pyplot as plt # Graph

# 데이터 가져오기
import pandas as pd
from sklearn import datasets

# 데이터 전처리
from sklearn.preprocessing import StandardScaler    # 연속 변수 표준화
from sklearn import preprocessing                   # 범주형 변수 수치화
from sklearn.preprocessing import LabelEncoder      # 범주형 변수 수치화

# 훈련/검증용 데이터 분리
from sklearn.model_selection import train_test_split    # 훈련과 테스트를 위한 데이터 분리

# 분류 모델
# from sklearn.tree import DecisionTreeClassifier         # 의사결정나무
# from sklearn.naive_bayes import GaussianNB            # 나이브 베이즈 분류
# from sklearn.neighbors import KNeighborsClassifier      # K-최근접 이웃
# from sklearn.ensemble import RandomForestClassifier   # 랜덤 포레스트
# from sklearn.linear_model import LogisticRegression     # 로지스틱 회귀분석
# from sklearn.svm import SVC                           # SVM(서포트벡터머신)
from sklearn.neural_network import MLPClassifier      # 다층 인공신경망
# from sklearn.ensemble import VotingClassifier           # 과반수 투표(Majority Voting) 
# from sklearn.ensemble import BaggingClassifier        # 배깅(Bagging) 
# from sklearn.ensemble import AdaBoostClassifier       # 부스팅(Boosting) 

# 모델 검정
from sklearn.metrics import confusion_matrix, classification_report # 정오분류표
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer  # 정확도, 민감도 등
from sklearn.metrics import roc_curve, roc_auc_score, auc   # ROC 곡선

# 최적화
from sklearn.model_selection import cross_validate, cross_val_score  # 교차 타당도
from sklearn.pipeline import make_pipeline  # 파이프라인 구축
from sklearn.model_selection import learning_curve, validation_curve # 학습곡선, 검증곡선
from sklearn.model_selection import GridSearchCV    # 하이퍼파라미터 튜닝

## 2) 데이터 가져오기

### 2-1) 데이터 프레임으로 저장
* csv 데이터를 dataframe으로 가져오기

In [ ]:
train = pd.read_csv('../Data/train.csv')

# train data의 상위 5개 출력
train.head()

### 2-2) 자료구조 살펴보기

In [ ]:
# null값 확인
print(train.info())

In [ ]:
# 배열의 행, 열 갯수 확인
print(train.shape)

In [ ]:
# column 값 확인
train.keys()

## 3) 데이터 전처리


In [ ]:
#범주형/연속형 변수 분리
cat_col = ['department','region', 'education', 'gender','recruitment_channel','awards_won?']
con_col = ['no_of_trainings', 'age', 'previous_year_rating',
            'length_of_service','avg_training_score']

### 3-1) ONE HOT ENCODING
- 범주형 변수는 먼저 숫자로 변경해주어야 한다.
- one-hot-encoding을 통해 string을 integer로 변환할 수 있다

In [ ]:
train_df = pd.get_dummies(train,columns=cat_col,drop_first=True)

In [ ]:
# train_df data의 상위 5개 출력
train_df.head()

### 3-2) 표준화
- 표준화는 train data를 표준화한 포멧을 test data에도 wjrdyd
- train data를 표준화할 때 사용한 평균과 표준편차를 test data에 적용하여 변환


In [ ]:
#연속형 변수 스케일링
scaler = StandardScaler()
scaler.fit(train.loc[:,con_col])
train.loc[:,con_col] = scaler.transform(train.loc[:,con_col])

### 3-3) null값 변경

In [ ]:
# null값을 평균 값으로 대체
train_df['previous_year_rating'].fillna(train_df['previous_year_rating'].mean(),inplace=True)

In [ ]:
# null값 반영 여부 확인
print(f'데이터 셋의 Null 값 개수:{train_df.isnull().sum().sum()}')

### 3-4) Data와 Target으로 분리
- 필요한 data만 추출
- data : `X`, taget : `y`로 분리

In [ ]:
# data
X = train_df.drop(['employee_id','is_promoted'], axis=1)
X.head()

In [ ]:
# target
y=train_df['is_promoted']
np.bincount(y)

## 4) 훈련 / 검증용 데이터 분할

### 4-1) 데이터 분리

In [ ]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y,
                     test_size = 0.3,   # test set의 비율
                     random_state = 1,  # 무작위 시드 번호
                     stratify = y)      # 결과 레이블의 비율대로 분리

## 5) 모델 구축

In [ ]:
mlp = MLPClassifier()

mlp.fit(X_train, y_train)

## 6) 모델 검정

### 6-1) 정오분류표

In [ ]:
mlp.fit(X_train, y_train)

In [ ]:
y_pred = mlp.predict(X_test)

In [ ]:
confmat = pd.DataFrame(confusion_matrix(y_test, y_pred),
                        index=['True[0]','True[1]'],
                        columns=['Predict[0]', 'Predict[1]'])
confmat

In [ ]:
# 정확도, 정밀도, 재현율, f1 score
print(f'잘못 분류된 샘플 개수: {(y_test != y_pred).sum()}')
print(f'정확도: {accuracy_score(y_test, y_pred):.3f}')
print(f'정밀도: {precision_score(y_true=y_test, y_pred=y_pred):.3f}')
print(f'재현율: {recall_score(y_true=y_test, y_pred=y_pred):.3f}')
print(f'F1: {f1_score(y_true=y_test, y_pred=y_pred):.3f}')

### 6-2) ROC 곡선

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, mlp.predict_proba(X_test)[:, 1])

plt.plot(fpr, tpr, '--', label="MLP")
plt.plot([0, 1], [0, 1], 'k--', label="random guess")
plt.plot([fpr], [tpr], 'r-', ms=10)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC')
plt.show()

## 7) 최적화

In [ ]:
pipe_mlp = make_pipeline(MLPClassifier())

In [ ]:
pipe_mlp.get_params().keys()

In [ ]:
train_sizes, train_scores, test_scores = \
    learning_curve(estimator=pipe_mlp, # 수정
                   X=X_train,
                   y=y_train,
                   train_sizes=np.linspace(0.1, 1.0, 10),
                   cv = 10,
                   n_jobs=1)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(train_sizes, train_mean,
         color='blue', marker='o',
         markersize=5, label='training accuracy')

plt.fill_between(train_sizes,
                 train_mean + train_std,
                 train_mean - train_std,
                 alpha=0.15, color='blue')

plt.plot(train_sizes, test_mean,
         color='green', marker='o',
         markersize=5, label='learning accuracy')

plt.fill_between(train_sizes,
                 test_mean + test_std,
                 test_mean - test_std,
                 alpha=0.15, color='green')

plt.grid()
plt.xlabel('Number of training samples')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.8, 1.0])
plt.tight_layout()
plt.show()

### 7-3) 검증 곡선으로 과대적합과 과소적합 조사하기
* 과대 적합 : 파라미터가 많음 -> 파라미터 축소
* 과소 적합 : 파라미터가 적음 -> 파라미터 추가

In [ ]:
param_range = [1e-06, 1e-05, 0.0001, 0.001]

train_scores, test_scores = \
    validation_curve(estimator=pipe_mlp, # 수정
                    X=X_train,
                    y=y_train,
                    param_name='mlpclassifier__alpha',
                    param_range=param_range,
                    cv = 10)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(param_range, train_mean,
         color='blue', marker='o',
         markersize=5, label='training accuracy')

plt.fill_between(param_range,
                 train_mean + train_std,
                 train_mean - train_std,
                 alpha=0.15, color='blue')

plt.plot(param_range, test_mean,
         color='green', marker='o',
         markersize=5, label='validation accuracy')

plt.fill_between(param_range,
                 test_mean + test_std,
                 test_mean - test_std,
                 alpha=0.15, color='green')

plt.grid()
plt.xlabel('Parameter C')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

### 7-4) 하이퍼 파라미터 튜닝

In [ ]:
param_range1 = [(5,5), (5,10), (10,5), (10,10)]
param_range2 = [1e-06, 1e-05, 1e-04, 1e-03]
param_grid = [{'mlpclassifier__hidden_layer_sizes': param_range1,
              'mlpclassifier__alpha': param_range2}]

gs = GridSearchCV(estimator=pipe_mlp,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=10,
                  n_jobs=-1)

gs = gs.fit(X_train, y_train)

print(f"Score : {gs.best_score_}")
print(f"Best Param: {gs.best_params_}")

## 8) 최적화 모델 검정

In [ ]:
best_mlp = gs.best_estimator_
best_mlp.fit(X_train, y_train)

In [ ]:
y_pred = mlp.predict(X_test)

In [ ]:
# y_pred = y_pred.idxmax(axis=1)

In [ ]:
confmat = pd.DataFrame(confusion_matrix(y_test, y_pred),
                       index=['True[0]','True[1]'],
                       columns=['Predict[0]', 'Predict[1]'])
confmat

In [ ]:
# 정확도, 정밀도, 재현율, f1 score
print(f'잘못 분류된 샘플 개수: {(y_test != y_pred).sum()}')
print(f'정확도: {accuracy_score(y_test, y_pred):.3f}')
print(f'정밀도: {precision_score(y_true=y_test, y_pred=y_pred):.3f}')
print(f'재현율: {recall_score(y_true=y_test, y_pred=y_pred):.3f}')
print(f'F1: {f1_score(y_true=y_test, y_pred=y_pred):.3f}')